# Multi-Agent Ticket Triage System

This notebook demonstrates how to build a **multi-agent system** using Azure AI Agent Service for automated ticket triage. The system uses multiple specialized agents that work together to analyze support tickets and determine:

- **Priority Level** (High/Medium/Low)
- **Team Assignment** (Frontend/Backend/Infrastructure/Marketing)
- **Effort Estimation** (Small/Medium/Large)

## Architecture Overview

The system consists of:
1. **Three Specialist Agents**: Each focused on one aspect of ticket analysis
2. **One Orchestrator Agent**: Uses the specialist agents as tools to provide comprehensive triage
3. **Connected Agent Tools**: Enable the orchestrator to call specialist agents as needed

## 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login --use-device-code
```

This will provide you with a device code and URL to authenticate in your browser, which is useful for:
- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.

## 📦 Import Required Libraries and Setup Environment

This cell imports all the necessary libraries for building our multi-agent system and loads environment variables from the `.env` file. We need:

- **Azure AI Agents SDK**: To create and manage multiple AI agents
- **Azure Identity**: For authentication with Azure services
- **Environment variables**: Project endpoint and model deployment details

In [ ]:
import os
from pathlib import Path
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import ConnectedAgentTool, MessageRole, ListSortOrder, ToolSet, FunctionTool
from azure.identity import InteractiveBrowserCredential
from dotenv import load_dotenv

# Load environment variables from parent .env
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir.parent / '.env')

# Get tenant ID and project connection string (not endpoint)
tenant_id = os.environ.get("TENANT_ID")
ai_foundry_project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")
model_deployment = os.environ.get("MODEL_DEPLOYMENT_NAME")

print(f"🔑 Using Tenant ID: {tenant_id}")
print(f"🔗 Project Connection: {ai_foundry_project_endpoint}")


## 🎯 Define Specialist Agent Instructions

Now we'll define the instructions for each of our three specialist agents. Each agent has a specific role in the ticket triage process.

### Priority Assessment Agent

This agent analyzes tickets to determine their urgency level. It categorizes tickets as High, Medium, or Low priority based on their impact on users and business operations.

In [ ]:
# Priority agent definition
priority_agent_name = "priority_agent"
priority_agent_instructions = """
Assess how urgent a ticket is based on its description.

Respond with one of the following levels:
- High: User-facing or blocking issues
- Medium: Time-sensitive but not breaking anything
- Low: Cosmetic or non-urgent tasks

Only output the urgency level and a very brief explanation.
"""

### Team Assignment Agent

This agent determines which team should handle each ticket based on the technical domain and expertise required. It assigns tickets to Frontend, Backend, Infrastructure, or Marketing teams.

In [ ]:
# Team agent definition
team_agent_name = "team_agent"
team_agent_instructions = """
Decide which team should own each ticket.

Choose from the following teams:
- Frontend
- Backend
- Infrastructure
- Marketing

Base your answer on the content of the ticket. Respond with the team name and a very brief explanation.
"""

### Effort Estimation Agent

This agent estimates the amount of work required to resolve each ticket. It categorizes the effort as Small (1 day), Medium (2-3 days), or Large (multi-day/cross-team effort).

In [ ]:
# Effort agent definition
effort_agent_name = "effort_agent"
effort_agent_instructions = """
Estimate how much work each ticket will require.

Use the following scale:
- Small: Can be completed in a day
- Medium: 2-3 days of work
- Large: Multi-day or cross-team effort

Base your estimate on the complexity implied by the ticket. Respond with the effort level and a brief justification.
"""

### Orchestrator Agent Instructions

The main orchestrator agent coordinates all three specialist agents. It receives the ticket and uses the specialist agents as tools to provide a comprehensive triage analysis.

In [ ]:
# Instructions for the primary agent
triage_agent_instructions = """
Triage the given ticket. Use the connected tools to determine the ticket's priority, 
which team it should be assigned to, and how much effort it may take.
"""

## 🔗 Connect to Azure AI Agent Service

This cell establishes a connection to the Azure AI Agent Service using our project endpoint and credentials. This client will be used to create and manage all our agents.

In [ ]:
# Connect to Azure AI Project using InteractiveBrowserCredential for consistency
credential = InteractiveBrowserCredential(tenant_id=tenant_id)
print("🌐 Launching interactive browser authentication (multi-agent notebook)...")
project_client = AIProjectClient(
    endpoint=ai_foundry_project_endpoint,
    credential=credential
)
print("✅ AIProjectClient initialized with InteractiveBrowserCredential")

## 🤖 Create Multi-Agent System

This cell creates all four agents in our system:

1. **Three Specialist Agents**: Priority, Team, and Effort assessment agents
2. **Connected Agent Tools**: Wrapper tools that allow the orchestrator to call specialists
3. **Main Orchestrator Agent**: Uses the specialist agents as tools for comprehensive ticket triage

Each specialist agent is created with its specific instructions and then wrapped in a `ConnectedAgentTool` so the orchestrator can use them.

In [ ]:
# FIXED: Create agents without context manager to keep client active
print("Creating agents...")

# Create the priority agent on the Azure AI agent service
priority_agent = project_client.agents.create_agent(
    model=model_deployment,
    name=priority_agent_name,
    instructions=priority_agent_instructions
)
print("✅ Priority agent created")

# Create a connected agent tool for the priority agent
priority_agent_tool = ConnectedAgentTool(
    id=priority_agent.id, 
    name=priority_agent_name, 
    description="Assess the priority of a ticket"
)

# Create the team agent and connected tool
team_agent = project_client.agents.create_agent(
    model=model_deployment,
    name=team_agent_name,
    instructions=team_agent_instructions
)
print("✅ Team agent created")

team_agent_tool = ConnectedAgentTool(
    id=team_agent.id, 
    name=team_agent_name, 
    description="Determines which team should take the ticket"
)

# Create the effort agent and connected tool
effort_agent = project_client.agents.create_agent(
    model=model_deployment,
    name=effort_agent_name,
    instructions=effort_agent_instructions
)
print("✅ Effort agent created")

effort_agent_tool = ConnectedAgentTool(
    id=effort_agent.id, 
    name=effort_agent_name, 
    description="Determines the effort required to complete the ticket"
)

# Create a main agent with the Connected Agent tools
agent = project_client.agents.create_agent(
    model=model_deployment,
    name="triage-agent",
    instructions=triage_agent_instructions,
    tools=[
        priority_agent_tool.definitions[0],
        team_agent_tool.definitions[0],
        effort_agent_tool.definitions[0]
    ]
)
print("✅ Main triage agent created with connected tools")
print("🎯 All agents are ready for use!")

## 💬 Create Conversation Thread

This cell creates a conversation thread that will contain our interaction with the multi-agent system. The thread maintains the context of our conversation and allows for back-and-forth communication.

In [ ]:
# Create thread for the chat session
print("Creating agent thread.")
thread = project_client.agents.threads.create()

## 🎯 Execute Multi-Agent Ticket Triage

Now we'll run our multi-agent ticket triage system! This cell will:

1. **Send a ticket**: Submit a sample support ticket to the orchestrator agent
2. **Agent coordination**: The orchestrator will call each specialist agent for their assessment
3. **Display results**: Show the complete conversation flow and final triage analysis

Watch as the orchestrator agent intelligently uses the three specialist agents to provide a comprehensive ticket analysis including priority, team assignment, and effort estimation.

In [ ]:
# Create the ticket prompt
prompt = "Users can't reset their password from the mobile app."

# Send a prompt to the agent
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content=prompt,
)
print(f"📝 Created user message: {prompt}")

# Create and process Agent run in thread with tools
print("🔄 Processing multi-agent thread. Please wait...")
run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)

if run.status == "failed":
    print(f"❌ Run failed: {run.last_error}")
else:
    print("✅ Multi-agent processing completed successfully")

# Fetch and analyze all messages
messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

print("\n📊 Multi-Agent Conversation Flow:")
print("=" * 50)

for message in messages:
    if message.text_messages:
        last_msg = message.text_messages[-1]
        print(f"🤖 {message.role.upper()}:")
        print(f"{last_msg.text.value}\n")

print("=" * 50)
print("🎯 Multi-agent ticket triage completed!")

## 🧹 Clean Up Resources

This cell deletes all the agents we created to avoid leaving resources running in Azure. It's important to clean up agents after use to prevent unnecessary costs.

In [ ]:
# Delete the agent when done
print("Cleaning up agents:")
project_client.agents.delete_agent(agent.id)
print("Deleted triage agent.")

# Delete the connected agents when done
project_client.agents.delete_agent(priority_agent.id)
print("Deleted priority agent.")
project_client.agents.delete_agent(team_agent.id)
print("Deleted team agent.")
project_client.agents.delete_agent(effort_agent.id)
print("Deleted effort agent.")